<a href="https://colab.research.google.com/github/joyceful-lee/AI-Tile-Placement/blob/master/project4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
import numpy as np

teamId = 1325
worldId = 0
url = "https://www.notexponential.com/aip2pgaming/api/rl/gw.php"


payload={}
headers = {
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36',
  'x-api-key': 'b65d58eeb9360bebb41e',
  'userid': '1096',
  'Content-Type': 'application/x-www-form-urlencoded'}

environment_rows = 41
environment_columns = 41

q_values = np.zeros((environment_rows, environment_columns, 4))
actions = ['N', 'S', 'E', 'W']
current_reward = -0.1
newState= None
x = 0
y = 0

#teamid 1330
def create_team(name):
  url = "https://www.notexponential.com/aip2pgaming/api/index.php"
  payload = {'type': 'team',
        'name': f'{name}',
        }
  files = [

    ]
  response = requests.request("POST", url, headers=headers, data=payload, files=files)
  print(response.text)

#create_team('prj4')

def add_meb(teamId):
  url = "https://www.notexponential.com/aip2pgaming/api/index.php"
  payload = {'type': 'member',
        'teamId': f'{teamId}',
        'userid': '1109'
        }
  files = [

    ]
  response = requests.request("POST", url, headers=headers, data=payload, files=files)
  print(response.text)
#add_meb(1331)
def get_runs(teamId, count):
  query_url = "https://www.notexponential.com/aip2pgaming/api/rl/score.php?type=runs&teamId={teamId}&count={count}"
  response = requests.request("GET", query_url, headers=headers, data=payload)
  data = json.loads(response.text)
  print(data)
#get_runs(1330, 0)

def getMyTeam():
    query_url = "https://www.notexponential.com/aip2pgaming/api/index.php?type=myTeams"
    response = requests.request("GET", query_url, headers=headers, data=payload)
    team_data = json.loads(response.text)
    team = team_data["myTeams"]
    print(f"Teams:{team}")
    return team
#getMyTeam()

def reset_team(teamId):
  query_url = f"https://www.notexponential.com/aip2pgaming/api/rl/reset.php?teamId={teamId}&otp=5712768807"
  response = requests.request("GET", query_url, headers=headers, data=payload)
  print(response.text)
#reset_team(1308)

def enter_world(teamId, worldId):
  payload = {'type': 'enter',
        'teamId': f'{teamId}',
        'worldId': f'{worldId}'}
  files = [

    ]
  response = requests.request("POST", url, headers=headers, data=payload, files=files)
  data = json.loads(response.text)
  print(data)
#enter_world(1308, 0)

def get_location(teamId):
  query_url = "https://www.notexponential.com/aip2pgaming/api/rl/gw.php?type=location&teamId={teamId}"
  response = requests.request("GET", query_url, headers=headers, data=payload)
  print(response.text)
#get_location(1308)

def get_state(teamId, move, worldId, x, y):
  global current_reward

  print(move)
  payload = {'type': 'move',
        'teamId': f'{teamId}',
          'move': move,
        'worldId': f'{worldId}'}
  files = [

    ]
  response = requests.request("POST", url, headers=headers, data=payload, files=files)
  data = json.loads(response.text)
  print(data)
  reward = data['reward']
  position = data['newState']
  print(reward)
  current_reward = reward
  current_pos = position
  print(position)
  if(position!= None):
    x=int(position['x'])
    y=int(position['y'])
    print(x)
    print(y)
  else:
    if(move=="N"):
      y +=1
    if(move=="S"):
      y -=1
    if(move=="E"):
      x +=1
    if(move=="W"):
      x -=1
  return x,y,reward

#get_runs(1309, 1)
#enter_world(1308,0)
#get_location(1307)
#enter_world(1309, 1)
#get_state(1308, "N", 0,0,39)




Win state reward is 10000.

Lose state reward is -10000.

The normal reward is -0.1


Reward:10000

{'code': 'OK', 'worldId': 0, 'runId': '6237', 'reward': 10000, 'scoreIncrement': 0, 'newState': None}


{'x': 0, 'y': '39'} +N
{'x': 1, 'y': '19'}
{'x': 39, 'y': '39'}+N
{'x': 19, 'y': '19'}+N

Reward:-10000

{'code': 'OK', 'worldId': 0, 'runId': '6537', 'reward': -10000, 'scoreIncrement': -1094.19, 'newState': None}

(1, 20, -10000)
(0,19,-10000)
(19,1,-10000)








In [ ]:
#check whether the state is a terminal state with reward feedback
def is_terminal(current_reward):
  #if the reward for this location is -0.1, then it is not a terminal state
  if current_reward == -0.1:
    return False
  else:
    return True

#Select the move action
def get_action(x,y):
  return np.argmax(q_values[x, y])


#get the position and reward info with the determined action
#return x,y,reward
def get_pos_and_reward(action,x,y):
  if actions[action]=="N":
    return get_state(teamId,"N",worldId,x,y)
  if actions[action]=="S":
    return get_state(teamId,"S",worldId,x,y)
  if actions[action]=="E":
    return get_state(teamId,"E",worldId,x,y)
  if actions[action]=="W":
    return get_state(teamId,"W",worldId,x,y)




In [ ]:
from matplotlib import pyplot
import os
import numpy as np
import random
import time

def update_grid(data, good_term_states, bad_term_states, obstacles, run_num, epoch, world, location):
    pyplot.figure(1)
    pyplot.clf()
    pyplot.imshow(data)
    pyplot.draw()
    pyplot.title(f'WORLD: {world} EPOCH: {epoch}')
    pyplot.ylim(-1, 41)
    pyplot.xlim(-1,41)
    for z in obstacles:
        pyplot.plot(z[0], z[1], marker="s", color = 'k')
    for x in good_term_states:
        pyplot.plot(x[0], x[1], marker="s", color = 'g')
    for y in bad_term_states:
        pyplot.plot(y[0], y[1], marker="s", color = 'r')
    pyplot.plot(location[0], location[1], marker="*", color = 'indigo')

    pyplot.show(block = False)
    pyplot.pause(0.0001)

    if not os.path.exists("./runs/world_{}/attempt_{}".format(world,run_num)):
        os.makedirs("./runs/world_{}/attempt_{}".format(world,run_num))
    pyplot.savefig("./runs/world_{}/attempt_{}/epoch_{}.png".format(world, run_num, epoch))

enter_world(1325,0)

#Q-learning parameters
current_reward=-0.1
discount_factor = 0.9
learning_rate = 0.9

#run through 1000 training episodes
for episode in range(1000):
  #get the starting location for this episode
  x, y = 0,0

  visited = [] #
  obstacles = []#
  bad_term_states = []
  good_term_states = []
  pyplot.figure(1, figsize=(10,10))
  curr_board = [[float('-inf')] * 40 for temp in range(40)]
  visited.append(tuple(x,y)) #

  learning_rate = learning_rate*discount_factor**(episode//100)
  while not is_terminal(current_reward):
    # select an action
    action = get_action(x, y)

    #store the current position
    current_x, current_y = x, y
    x, y, current_reward = get_pos_and_reward(action,current_x, current_y)

    if(current_reward >=0):
        good_term_states.append(tuple(current_x,current_y))
    else:
        bad_term_states.append(tuple(current_x,current_y))

    current_q_value = q_values[current_x, current_y, action]
    temporal_difference = current_reward + (discount_factor * np.max(q_values[x, y])) - current_q_value

    curr_board[current_x][current_y] = 1
    for i in range (len(curr_board)):
        for j in range(len(curr_board)):
            if (curr_board[i][j] != 0):
                curr_board[i][j] -= .1
    for obstacle in obstacles:
        if obstacle in visited:
            obstacles.remove(obstacle)
    update_grid(curr_board, good_term_states, bad_term_states, obstacles, epoch, worldId, tuple(x,y))


    #update the q-value
    new_q_value = current_q_value + (learning_rate * temporal_difference)
    q_values[current_x, current_y, action] = new_q_value
print('Complete learning.')





{'code': 'FAIL', 'message': 'Cannot enter the world.  You are currently in world: 0'}


TypeError: ignored

<Figure size 720x720 with 0 Axes>

In [ ]:

#play in the world with the trained Q-value table
"""def play():
    x, y = 0, 0
    current_reward=-0.1
    path = []
    path.append([x, y])
    #keep moving until it is terminal
    while not is_terminal(current_reward):
      #get the best action
      action = get_action(x, y)
      #move to the new state with the best action
      x, y, current_reward = get_pos_and_reward(action,x,y)
      #add the new position to path
      path.append([x, y])
    return path


#enter_world(1308,0)
#print(play())
get_state(1308,"N",0,39,39)"""


'def play():\n    x, y = 0, 0\n    current_reward=-0.1\n    path = []\n    path.append([x, y])\n    #keep moving until it is terminal\n    while not is_terminal(current_reward):\n      #get the best action\n      action = get_action(x, y)\n      #move to the new state with the best action\n      x, y, current_reward = get_pos_and_reward(action,x,y)\n      #add the new position to path\n      path.append([x, y])\n    return path\n\n\n#enter_world(1308,0)\n#print(play())\nget_state(1308,"N",0,39,39)'